# Driver Splits

Script to rebase the split times for a stage and display them relative to a specified driver.

The intention is to generate a report on a stage that is meaningful to a specified driver.

Ideally the report should:

- show where the driver finished on the stage (stage rank)
- show the running stage delta at each split compared to each other driver
- show the extent to which a driver gained or lost time on each split compared to each other driver
- show the start order (so that this can be related to stage rank)
- identify the overall position at the end of the stage for each driver
- show whether overall positions were gained or lost after the stage (not implemented yet; need a +=- column)

In [436]:
import notebookimport

if __name__=='__main__':
    typ = 'overall' #this defines ???
    #rebase='overallleader' #TO DO
    rebase='OGI'#'PAD'
    MAXINSPLITDELTA=20 #set xlim on the within split delta
    ss='SS13'
    
    #The drivercode inbuilds some intelligence
    drivercode=rebase

In [321]:
sr = __import__("Charts - Stage Results")
ssd = __import__("Charts - Split Sector Delta")

In [322]:
#!pip3 install pytablewriter

In [323]:
import os
import sqlite3
import pandas as pd
import pytablewriter
import six
from numpy import NaN

#dbname='wrc18.db'
#dbname='france18.db'
#conn = sqlite3.connect(dbname)

if __name__=='__main__':
    #dbname='wrc18.db'
    dbname='spain18.db'
    conn = sqlite3.connect(dbname)
    rally='Spain'
    rc='RC1'
    year=2018
    #ss='SS4'

In [324]:
if __name__=='__main__':
    #This doesn't appear to be used elsewhere in this notebook
    #May support logic for checking stage status?
    stageresults = sr.dbGetRallyStages(conn, rally).sort_values('number')
    stageresults.head()

,legDate,date,startListId,status,itineraryLegId,itinerarySectionId,section,order,code,distance,eventId,name,number,stageId,stageType,status,timingPrecision,itineraryLegId,itinerarySections.itinerarySectionId
0,2018-10-25,Thursday 25th October,236.0,Completed,143,328,Section 2,2,SS1,3.20,37,Barcelona (asphalt) TV Live,1,823,SpecialStage,Completed,Tenth,143,328
1,2018-10-26,Friday 26th October,234.0,Completed,142,329,Section 3,3,SS2,7.00,37,Gandesa 1 (gravel),2,828,SpecialStage,Completed,Tenth,142,329
2,2018-10-26,Friday 26th October,234.0,Completed,142,329,Section 3,3,SS3,26.59,37,Pesells 1 (gravel),3,830,SpecialStage,Completed,Tenth,142,329
3,2018-10-26,Friday 26th October,234.0,Completed,142,329,Section 3,3,SS4,38.85,37,La Fatarella -Vilalba 1 (gravel & asphalt),4,831,SpecialStage,Interrupted,Tenth,142,329
6,2018-10-26,Friday 26th October,234.0,Completed,142,330,Section 4,4,SS5,7.00,37,Gandesa 2 (gravel),5,835,SpecialStage,Completed,Tenth,142,330


In [325]:
def rebaseOverallRallyTime(stagerank_overall, drivercode):
    ''' Rebase overall stage rank relative to a specified driver. '''
    
    #TO DO - need to make sure this is group applied to each stage
    rebase = stagerank_overall[stagerank_overall['driver.code']==drivercode][['code','totalTimeMs']].set_index('code').to_dict(orient='dict')['totalTimeMs']
    stagerank_overall['rebased'] = stagerank_overall['totalTimeMs'] - stagerank_overall['code'].map(rebase)
    return stagerank_overall

In [424]:
if __name__=='__main__':
    stagerank_overall = sr.getEnrichedStageRank(conn, rally, typ='overall')
    stagerank_overall[stagerank_overall['code']==ss][['driver.code','position','totalTimeMs','code']]

In [655]:
def rebased_stage_stagegerank(conn,rally,ss,drivercode,typ='overall'):
    _stagerank_overall = sr.getEnrichedStageRank(conn, rally, typ=typ)
    zz=rebaseOverallRallyTime(stagerank_overall, drivercode)#, ss)
    #Get the rebased times for a particular stage
    zz=zz[zz['code']==ss][['driver.code','position','totalTimeMs','code', 'rebased']].set_index('driver.code')
    #Scale down the time from milliseconds to seconds
    zz['Overall Time']=-zz['rebased']/1000
    return zz

In [656]:
if __name__=='__main__':
    zz=rebased_stage_stagegerank(conn,rally,ss, drivercode)
    display(zz)

,position,totalTimeMs,code,rebased,Overall Time
driver.code,,,,,
LAT,1,9136500,SS13,-7200,7.2
OGI,2,9143700,SS13,0,0.0
LOE,3,9146400,SS13,2700,-2.7
EVA,4,9147700,SS13,4000,-4.0
NEU,5,9150500,SS13,6800,-6.8
SOR,6,9154100,SS13,10400,-10.4
LAP,7,9178100,SS13,34400,-34.4
TÄN,8,9200600,SS13,56900,-56.9
BRE,9,9234300,SS13,90600,-90.6


In [427]:
if __name__=='__main__':
    display(stagerank_overall.columns)

Index(['diffFirst', 'diffFirstMs', 'diffPrev', 'diffPrevMs', 'entryId',
       'penaltyTime', 'penaltyTimeMs', 'position', 'stageTime', 'stageTimeMs',
       'totalTime', 'totalTimeMs', 'stageId', 'class', 'code', 'distance',
       'name', 'snum', 'driver.code', 'entrant.name', 'classrank',
       'gainedClassPos', 'gainedClassLead', 'classPosDiff', 'lostClassLead',
       'retainedClassLead', 'gainedOverallPos', 'gainedOverallLead',
       'overallPosDiff', 'lostOverallLead', 'retainedOverallLead', 'stagewin',
       'stagewincount', 'winsinarow', 'gainedTime', 'rebased'],
      dtype='object')

In [329]:
if __name__=='__main__':
    #Preview a long format dataframe describing position and stage code for a specified driver
    #This appears not be be referenced anywhere else in this notebook
    stagerank_stage = sr.getEnrichedStageRank(conn, rally, typ='stage')
    stagerank_stage[stagerank_stage['driver.code']==rebase][['position','code']]

,position,code
0,1.0,SS1
19,6.0,SS2
37,10.0,SS3
47,6.0,SS4
63,8.0,SS5
78,9.0,SS6
88,5.0,SS7
99,NaN,SS8
116,4.0,SS9
130,4.0,SS10


In [330]:
if __name__=='__main__':
    splits = ssd.dbGetSplits(conn,rally,ss,rc)

    elapseddurations=ssd.getElapsedDurations(splits)
    elapseddurations.head()

,drivercode,elapsedDurationS,startDateTime,section
0,AL,214.5,2018-10-27T13:08:00,1
1,AL,347.9,2018-10-27T13:08:00,2
2,AL,518.8,2018-10-27T13:08:00,3
3,AL,573.4,2018-10-27T13:08:00,4
4,AL,668.4,2018-10-27T13:08:00,5


In [429]:
def getRoadPosition(splits):
    ''' Get road position for each driver for a given stage. '''
    roadPos=splits[['drivercode','startDateTime']].drop_duplicates()
    roadPos = roadPos.set_index('drivercode')
    roadPos['Road Position']=roadPos['startDateTime'].rank().astype(int).astype(str)
    return roadPos

In [430]:
if __name__=='__main__':
    roadPos = getRoadPosition(splits)
    display(roadPos)

,startDateTime,Road Position
drivercode,,
AL,2018-10-27T13:08:00,1
SUN,2018-10-27T13:11:00,2
LAP,2018-10-27T13:14:00,3
NEU,2018-10-27T13:17:00,4
BRE,2018-10-27T13:20:00,5
OGI,2018-10-27T13:23:00,6
MIK,2018-10-27T13:26:00,7
LAT,2018-10-27T13:29:00,8
LOE,2018-10-27T13:32:00,9


In [444]:
if __name__=='__main__':
    rebasedelapseddurations = ssd.rebaseElapsedDurations(elapseddurations, drivercode)
    display(rebasedelapseddurations.head())

,drivercode,elapsedDurationS,startDateTime,section,rebased
0,AL,214.5,2018-10-27T13:08:00,1,31.6
1,AL,347.9,2018-10-27T13:08:00,2,49.2
2,AL,518.8,2018-10-27T13:08:00,3,77.5
3,AL,573.4,2018-10-27T13:08:00,4,86.7
4,AL,668.4,2018-10-27T13:08:00,5,97.4


In [443]:
def pivotRebasedElapsedDurations(rebasedelapseddurations, ss):
    ''' Pivot rebased elapsed durations (that is, deltas relative target).
        Rows give stage delta at each split for a specific driver. '''
    rbe=-rebasedelapseddurations.pivot('drivercode','section','rebased')
    rbe.columns=list(rbe.columns)[:-1]+['{} Overall'.format(ss)]
    rbe=rbe.sort_values(rbe.columns[-1],ascending =False)
    return rbe

if __name__=='__main__':
    rbe = pivotRebasedElapsedDurations(rebasedelapseddurations, ss)
    display(rbe)

,1,2,3,4,5,6,SS13 Overall
drivercode,,,,,,,
NEU,1.5,-0.0,1.3,1.0,1.2,1.3,0.9
OGI,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
SUN,-1.4,-2.0,-1.6,-2.6,-2.3,-3.9,-5.7
EVA,-2.5,-3.6,-6.0,-6.1,-6.4,-6.7,-7.6
LAT,-1.4,-2.3,-6.4,-7.0,-6.5,-7.8,-8.2
LOE,-4.2,-5.5,-6.5,-7.6,-9.2,-9.7,-10.6
TÄN,2.2,2.0,-9.7,-10.3,-9.6,-11.0,-11.5
LAP,1.4,3.3,5.9,-12.7,-11.9,-13.1,-13.9
BRE,-2.5,-5.4,-7.8,-9.1,-10.2,-13.0,-14.4


In [335]:
#https://pandas.pydata.org/pandas-docs/stable/style.html
def color_negative(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if isinstance(val, str): return ''
    elif val and (isinstance(val,int) or isinstance(val,float)):
        color = 'green' if val < 0 else 'red' if val > 0  else 'black'
    else:
        color='white'
    return 'color: %s' % color

In [445]:
if __name__=='__main__':
    #test of applying style to pandas dataframe
    s = rbe.style.applymap(color_negative)
    display(s)

,1,2,3,4,5,6,SS13 Overall
drivercode,,,,,,,
NEU,1.5,-0,1.3,1,1.2,1.3,0.9
OGI,-0,-0,-0,-0,-0,-0,-0
SUN,-1.4,-2,-1.6,-2.6,-2.3,-3.9,-5.7
EVA,-2.5,-3.6,-6,-6.1,-6.4,-6.7,-7.6
LAT,-1.4,-2.3,-6.4,-7,-6.5,-7.8,-8.2
LOE,-4.2,-5.5,-6.5,-7.6,-9.2,-9.7,-10.6
TÄN,2.2,2,-9.7,-10.3,-9.6,-11,-11.5
LAP,1.4,3.3,5.9,-12.7,-11.9,-13.1,-13.9
BRE,-2.5,-5.4,-7.8,-9.1,-10.2,-13,-14.4


In [ ]:
# TO DO:
# - calculate stage position at each split
# - calculate rank within that sector

In [337]:
if __name__=='__main__':
    #splitdurations are the time in each sector (time take to get from one split to the next)
    splitdurations = ssd.getSplitDurationsFromSplits(conn,rally,ss,rc)
    splitdurations.head()

,drivercode,splitDurationS,startDateTime,stageTimeDurationMs,section
0,AL,214.5,2018-10-27T13:08:00,909400.0,1
1,AL,133.4,2018-10-27T13:08:00,909400.0,2
2,AL,170.9,2018-10-27T13:08:00,909400.0,3
3,AL,54.6,2018-10-27T13:08:00,909400.0,4
4,AL,95.0,2018-10-27T13:08:00,909400.0,5


In [338]:
if __name__=='__main__':
    rebasedSplits = ssd.rebaseSplitDurations(splitdurations, drivercode)
    rebasedSplits.head()

,drivercode,splitDurationS,startDateTime,stageTimeDurationMs,section,rebased
0,AL,214.5,2018-10-27T13:08:00,909400.0,1,31.6
1,AL,133.4,2018-10-27T13:08:00,909400.0,2,17.6
2,AL,170.9,2018-10-27T13:08:00,909400.0,3,28.3
3,AL,54.6,2018-10-27T13:08:00,909400.0,4,9.2
4,AL,95.0,2018-10-27T13:08:00,909400.0,5,10.7


In [442]:
if __name__=='__main__':
    #preview what's available as a splitduration
    display(splitdurations[splitdurations['drivercode'].isin( ['PAD','NEU'])])

,drivercode,splitDurationS,startDateTime,stageTimeDurationMs,section,rebased
18,NEU,181.4,2018-10-27T13:17:00,787300.0,1,-1.5
19,NEU,117.3,2018-10-27T13:17:00,787300.0,2,1.5
20,NEU,141.3,2018-10-27T13:17:00,787300.0,3,-1.3
21,NEU,45.7,2018-10-27T13:17:00,787300.0,4,0.3
22,NEU,84.1,2018-10-27T13:17:00,787300.0,5,-0.2
23,NEU,168.4,2018-10-27T13:17:00,787300.0,6,0.0
79,NEU,49.2,2018-10-27T13:17:00,787300.0,7,0.4


In [447]:
def pivotRebasedSplits(rebasedSplits):
    ''' For each driver row, find the split '''
    rbp=-rebasedSplits.pivot('drivercode','section','rebased')
    rbp.columns=['D{}'.format(c) for c in rbp.columns]
    rbp.sort_values(rbp.columns[-1],ascending =True)
    return rbp

if __name__=='__main__':
    rbp = pivotRebasedSplits(rebasedSplits)
    display(rbp)

,D1,D2,D3,D4,D5,D6,D7
drivercode,,,,,,,
AL,-31.6,-17.6,-28.3,-9.2,-10.7,-18.6,-5.2
BRE,-2.5,-2.9,-2.4,-1.4,-1.0,-2.9,-1.4
EVA,-2.5,-1.1,-2.4,-0.1,-0.3,-0.2,-0.9
LAP,1.4,1.9,2.5,-18.5,0.7,-1.2,-0.8
LAT,-1.4,-0.9,-4.1,-0.6,0.6,-1.3,-0.4
LOE,-4.2,-1.3,-1.0,-1.1,-1.6,-0.5,-0.9
MIK,-3.5,-2.9,-4.8,-2.7,-0.9,-3.5,-2.1
NEU,1.5,-1.5,1.3,-0.3,0.2,-0.0,-0.4
OGI,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0


In [341]:
if __name__=='__main__':
    #Just remind ourselves of what is available in the road position data
    display(roadPos)

,startDateTime,Road Position
drivercode,,
AL,2018-10-27T13:08:00,1
SUN,2018-10-27T13:11:00,2
LAP,2018-10-27T13:14:00,3
NEU,2018-10-27T13:17:00,4
BRE,2018-10-27T13:20:00,5
OGI,2018-10-27T13:23:00,6
MIK,2018-10-27T13:26:00,7
LAT,2018-10-27T13:29:00,8
LOE,2018-10-27T13:32:00,9


In [574]:
def getDriverSplitReportBaseDataframe(rbe,rbp, zz, roadPos):
    ''' Create a base dataframe for the rebased driver split report. '''
    rb2=pd.merge(rbe,rbp,left_index=True, right_index=True)
    rb2=pd.merge(rb2,zz[['position','Overall Time']],left_index=True, right_index=True)
    rb2=pd.merge(rb2,roadPos[['Road Position']],left_index=True, right_index=True)
    cols=rb2.columns.tolist()
    rb2=rb2[[cols[-1]]+cols[:-1]]
    return rb2

if __name__=='__main__':
    rb2=getDriverSplitReportBaseDataframe(rbe,rbp, zz, roadPos)
    display(rb2)

,Road Position,1,2,3,4,5,6,SS13 Overall,D1,D2,D3,D4,D5,D6,D7,position,Overall Time
NEU,4,1.5,-0.0,1.3,1.0,1.2,1.3,0.9,1.5,-1.5,1.3,-0.3,0.2,-0.0,-0.4,5,-6.8
OGI,6,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,2,0.0
SUN,2,-1.4,-2.0,-1.6,-2.6,-2.3,-3.9,-5.7,-1.4,-0.6,0.4,-1.0,0.3,-1.6,-1.8,11,-182.0
EVA,10,-2.5,-3.6,-6.0,-6.1,-6.4,-6.7,-7.6,-2.5,-1.1,-2.4,-0.1,-0.3,-0.2,-0.9,4,-4.0
LAT,8,-1.4,-2.3,-6.4,-7.0,-6.5,-7.8,-8.2,-1.4,-0.9,-4.1,-0.6,0.6,-1.3,-0.4,1,7.2
LOE,9,-4.2,-5.5,-6.5,-7.6,-9.2,-9.7,-10.6,-4.2,-1.3,-1.0,-1.1,-1.6,-0.5,-0.9,3,-2.7
TÄN,12,2.2,2.0,-9.7,-10.3,-9.6,-11.0,-11.5,2.2,-0.2,-11.7,-0.6,0.7,-1.4,-0.5,8,-56.9
LAP,3,1.4,3.3,5.9,-12.7,-11.9,-13.1,-13.9,1.4,1.9,2.5,-18.5,0.7,-1.2,-0.8,7,-34.4
BRE,5,-2.5,-5.4,-7.8,-9.1,-10.2,-13.0,-14.4,-2.5,-2.9,-2.4,-1.4,-1.0,-2.9,-1.4,9,-90.6
MIK,7,-3.5,-6.4,-11.2,-13.9,-14.8,-18.3,-20.4,-3.5,-2.9,-4.8,-2.7,-0.9,-3.5,-2.1,10,-121.1


In [542]:
if __name__=='__main__':
    display(rb2.dtypes)

Road Position     object
1                float64
2                float64
3                float64
4                float64
5                float64
6                float64
SS13 Overall     float64
D1               float64
D2               float64
D3               float64
D4               float64
D5               float64
D6               float64
D7               float64
position           int64
Overall Time     float64
dtype: object

In [580]:
#There seems to be missing tenths?
#Elapsed durations are provided in milliseconds. Need to round correctly to tenths?
#Elapsed times grabbed from ssd.dbGetSplits(conn,rally,ss,rc)

def cleanDriverSplitReportBaseDataframe(rb2, ss):
    rb2=rb2.replace(0,NaN)
    #rb2=rb2.fillna('') #This casts columns containing NA to object type
    rb2['position']=rb2['position']#.astype(str) #Cast as str so we don't highlight colour
    rb2.rename(columns={'position': 'Overall Position'}, inplace=True)
    
    #The following is calculated rather than being based on the actual timing data / result for the previous stage
    #Would be better to explicitly grab data for previous stage, along with previous ranking
    rb2['Previous'] =  rb2['Overall Time'] - rb2['{} Overall'.format(ss)]
    #Related to this, would be useful to have an overall places gained / lost column
    
    rb2['Road Position']=rb2['Road Position'].astype(float)
    return rb2

def styleDriverSplitReportBaseDataframe(rb2, ss):
    ''' First attempt. THIS IS DEPRECATED. '''
    s=rb2.fillna('').style.applymap(color_negative,
                                           subset=[c for c in rb2.columns if isinstance(c, int) and c not in ['Overall Position', 'Road Position']])
    #data.style.applymap(highlight_cols, subset=pd.IndexSlice[:, ['B', 'C']])

    s.set_caption("{}: running split times and deltas within each split.".format(ss))
    return s
    
if __name__=='__main__':
    rb2c = cleanDriverSplitReportBaseDataframe(rb2.copy(), ss)
    s = styleDriverSplitReportBaseDataframe(rb2c, ss)

In [581]:
from IPython.core.display import HTML

if __name__=='__main__':
    html=s.render()
    display(HTML(html))

,Road Position,1,2,3,4,5,6,SS13 Overall,D1,D2,D3,D4,D5,D6,D7,Overall Position,Overall Time,Previous
NEU,4,1.5,,1.3,1,1.2,1.3,0.9,1.5,-1.5,1.3,-0.3,0.2,,-0.4,5,-6.8,-7.7
OGI,6,,,,,,,,,,,,,,,2,,
SUN,2,-1.4,-2,-1.6,-2.6,-2.3,-3.9,-5.7,-1.4,-0.6,0.4,-1,0.3,-1.6,-1.8,11,-182,-176.3
EVA,10,-2.5,-3.6,-6,-6.1,-6.4,-6.7,-7.6,-2.5,-1.1,-2.4,-0.1,-0.3,-0.2,-0.9,4,-4,3.6
LAT,8,-1.4,-2.3,-6.4,-7,-6.5,-7.8,-8.2,-1.4,-0.9,-4.1,-0.6,0.6,-1.3,-0.4,1,7.2,15.4
LOE,9,-4.2,-5.5,-6.5,-7.6,-9.2,-9.7,-10.6,-4.2,-1.3,-1,-1.1,-1.6,-0.5,-0.9,3,-2.7,7.9
TÄN,12,2.2,2,-9.7,-10.3,-9.6,-11,-11.5,2.2,-0.2,-11.7,-0.6,0.7,-1.4,-0.5,8,-56.9,-45.4
LAP,3,1.4,3.3,5.9,-12.7,-11.9,-13.1,-13.9,1.4,1.9,2.5,-18.5,0.7,-1.2,-0.8,7,-34.4,-20.5
BRE,5,-2.5,-5.4,-7.8,-9.1,-10.2,-13,-14.4,-2.5,-2.9,-2.4,-1.4,-1,-2.9,-1.4,9,-90.6,-76.2
MIK,7,-3.5,-6.4,-11.2,-13.9,-14.8,-18.3,-20.4,-3.5,-2.9,-4.8,-2.7,-0.9,-3.5,-2.1,10,-121.1,-100.7


In [634]:
from math import nan
def bg_color(s):
    '''
    Set background colour based on .
    '''
    red='#d65f5f'
    green='#5fba7d'
    attrs=[]
    for _s in s:
        if _s < 0:
            attr = 'background-color: green; color: white'
        elif _s > 0: 
            attr = 'background-color: red; color: white'
        else:
            attr = ''
        attrs.append(attr)
    return attrs

In [635]:
import seaborn as sns

def moreStyleDriverSplitReportBaseDataframe(rb2,ss):
    #https://community.modeanalytics.com/gallery/python_dataframe_styling/
    # Set CSS properties for th elements in dataframe
    th_props = [
      ('font-size', '11px'),
      ('text-align', 'center'),
      ('font-weight', 'bold'),
      ('color', '#6d6d6d'),
      ('background-color', '#f7f7f9')
      ]

    # Set CSS properties for td elements in dataframe
    td_props = [
      ('font-size', '11px')
      ]

    # Set table styles
    styles = [
      dict(selector="th", props=th_props),
      dict(selector="td", props=td_props)
      ]

    #.format({'B': "{:0<4.0f}", 'D': '{:+.2f}'})
    cmg = sns.light_palette("green", as_cmap=True)
    cm=sns.light_palette((210, 90, 60), input="husl",as_cmap=True)
    #(df.style
    s2=(rb2.style.applymap(color_negative,
                           subset=[c for c in rb2.columns if isinstance(c, int) and c not in ['Overall Position', 'Road Position']])
      .background_gradient(cmap=cm, subset=['Road Position' ])
      .highlight_min(subset=['Overall Position'], color='lightgrey')
      .highlight_max(subset=['Overall Time'], color='lightgrey')
      .highlight_max(subset=['Previous'], color='lightgrey')
      .apply(bg_color,subset=['{} Overall'.format(ss)],)
      .bar(subset=[c for c in rb2.columns if str(c).startswith('D')], align='zero', color=[ '#5fba7d','#d65f5f'])
      #.bar(subset=[c for c in rb2.columns if isinstance(c, int)], align='mid', color=[ '#5fba7d','#d65f5f'])
      .set_table_styles(styles)
    )

      #.set_caption('This is a custom caption.')
      #.format({'total_amt_usd_pct_diff': "{:.2%}"})
    
    #nan issue: https://github.com/pandas-dev/pandas/issues/21527
    return s2.render().replace('nan','')

if __name__=='__main__':
    rb2c = cleanDriverSplitReportBaseDataframe(rb2.copy(), ss)
    s2 = moreStyleDriverSplitReportBaseDataframe(rb2c,ss)
    display(HTML(s2))

,Road Position,1,2,3,4,5,6,SS13 Overall,D1,D2,D3,D4,D5,D6,D7,Overall Position,Overall Time,Previous
NEU,4,1.5,,1.3,1,1.2,1.3,0.9,1.5,-1.5,1.3,-0.3,0.2,,-0.4,5,-6.8,-7.7
OGI,6,,,,,,,,,,,,,,,2,,
SUN,2,-1.4,-2,-1.6,-2.6,-2.3,-3.9,-5.7,-1.4,-0.6,0.4,-1,0.3,-1.6,-1.8,11,-182,-176.3
EVA,10,-2.5,-3.6,-6,-6.1,-6.4,-6.7,-7.6,-2.5,-1.1,-2.4,-0.1,-0.3,-0.2,-0.9,4,-4,3.6
LAT,8,-1.4,-2.3,-6.4,-7,-6.5,-7.8,-8.2,-1.4,-0.9,-4.1,-0.6,0.6,-1.3,-0.4,1,7.2,15.4
LOE,9,-4.2,-5.5,-6.5,-7.6,-9.2,-9.7,-10.6,-4.2,-1.3,-1,-1.1,-1.6,-0.5,-0.9,3,-2.7,7.9
TÄN,12,2.2,2,-9.7,-10.3,-9.6,-11,-11.5,2.2,-0.2,-11.7,-0.6,0.7,-1.4,-0.5,8,-56.9,-45.4
LAP,3,1.4,3.3,5.9,-12.7,-11.9,-13.1,-13.9,1.4,1.9,2.5,-18.5,0.7,-1.2,-0.8,7,-34.4,-20.5
BRE,5,-2.5,-5.4,-7.8,-9.1,-10.2,-13,-14.4,-2.5,-2.9,-2.4,-1.4,-1,-2.9,-1.4,9,-90.6,-76.2
MIK,7,-3.5,-6.4,-11.2,-13.9,-14.8,-18.3,-20.4,-3.5,-2.9,-4.8,-2.7,-0.9,-3.5,-2.1,10,-121.1,-100.7


In [661]:
def getDriverSplitsReport(conn, rally, ss, drivercode, rc='RC1', typ='overall', order=None):
    zz = rebased_stage_stagegerank(conn,rally,ss,drivercode, typ=typ)
    splits = ssd.dbGetSplits(conn,rally,ss,rc)
    elapseddurations=ssd.getElapsedDurations(splits)
    roadPos = getRoadPosition(splits)
    rebasedelapseddurations = ssd.rebaseElapsedDurations(elapseddurations, drivercode)
    rbe = pivotRebasedElapsedDurations(rebasedelapseddurations, ss)
    
    #splitdurations are the time in each sector (time take to get from one split to the next)
    splitdurations = ssd.getSplitDurationsFromSplits(conn,rally,ss,rc)
    rebasedSplits = ssd.rebaseSplitDurations(splitdurations, drivercode)
    rbp = pivotRebasedSplits(rebasedSplits)
    
    rb2=getDriverSplitReportBaseDataframe(rbe,rbp, zz, roadPos)
    rb2 = cleanDriverSplitReportBaseDataframe(rb2, ss)
    if order=='overall':
        rb2=rb2.sort_values('Overall Position', ascending=True)
    elif order=='previous':
        rb2=rb2.fillna(0).sort_values('Previous', ascending=False).replace(0,NaN)
    elif order=='roadpos':
        rb2=rb2.sort_values('Road Position', ascending=True)
    #s = styleDriverSplitReportBaseDataframe(rb2, ss)
    s2 = moreStyleDriverSplitReportBaseDataframe(rb2,ss)
    return s2

if __name__=='__main__':
    s2 = getDriverSplitsReport(conn, rally, ss, drivercode, rc, typ)
    display(HTML(s2))

,Road Position,1,2,3,4,5,6,SS13 Overall,D1,D2,D3,D4,D5,D6,D7,Overall Position,Overall Time,Previous
NEU,4,1.5,,1.3,1,1.2,1.3,0.9,1.5,-1.5,1.3,-0.3,0.2,,-0.4,5,-6.8,-7.7
OGI,6,,,,,,,,,,,,,,,2,,
SUN,2,-1.4,-2,-1.6,-2.6,-2.3,-3.9,-5.7,-1.4,-0.6,0.4,-1,0.3,-1.6,-1.8,11,-182,-176.3
EVA,10,-2.5,-3.6,-6,-6.1,-6.4,-6.7,-7.6,-2.5,-1.1,-2.4,-0.1,-0.3,-0.2,-0.9,4,-4,3.6
LAT,8,-1.4,-2.3,-6.4,-7,-6.5,-7.8,-8.2,-1.4,-0.9,-4.1,-0.6,0.6,-1.3,-0.4,1,7.2,15.4
LOE,9,-4.2,-5.5,-6.5,-7.6,-9.2,-9.7,-10.6,-4.2,-1.3,-1,-1.1,-1.6,-0.5,-0.9,3,-2.7,7.9
TÄN,12,2.2,2,-9.7,-10.3,-9.6,-11,-11.5,2.2,-0.2,-11.7,-0.6,0.7,-1.4,-0.5,8,-56.9,-45.4
LAP,3,1.4,3.3,5.9,-12.7,-11.9,-13.1,-13.9,1.4,1.9,2.5,-18.5,0.7,-1.2,-0.8,7,-34.4,-20.5
BRE,5,-2.5,-5.4,-7.8,-9.1,-10.2,-13,-14.4,-2.5,-2.9,-2.4,-1.4,-1,-2.9,-1.4,9,-90.6,-76.2
MIK,7,-3.5,-6.4,-11.2,-13.9,-14.8,-18.3,-20.4,-3.5,-2.9,-4.8,-2.7,-0.9,-3.5,-2.1,10,-121.1,-100.7


In [658]:
if __name__=='__main__':
    s2 = getDriverSplitsReport(conn, rally, 'SS11', 'LAT', rc, typ)
    display(HTML(s2))

,Road Position,1,2,3,SS11 Overall,D1,D2,D3,D4,Overall Position,Overall Time,Previous
NEU,4,0.3,2,3.3,2.6,0.3,1.6,1.4,-0.7,6,-29.5,-32.1
TÄN,12,-0.6,-0.3,0.1,1.4,-0.6,0.3,0.4,1.3,8,-60.4,-61.8
LAP,3,-2.1,-0.7,-0.8,0.2,-2.1,1.3,-0.1,1,7,-37.6,-37.8
LAT,8,,,,,,,,,1,,
OGI,6,-2.1,-1.7,-1.8,-1.9,-2.1,0.4,-0.1,-0.1,5,-14.7,-12.8
EVA,10,-0.7,-2,-3,-3,-0.7,-1.3,-1,,3,-11.3,-8.3
BRE,5,-1.6,-2,-2.1,-3.1,-1.6,-0.4,-0.1,-1,9,-62.1,-59
LOE,9,-1.3,-3.7,-2.9,-3.7,-1.3,-2.5,0.9,-0.8,4,-14.5,-10.8
SOR,11,-0.1,-2.6,-2.7,-4.2,-0.1,-2.5,-0.1,-1.5,2,-3.9,0.3
SUN,2,-4.2,-6.5,-7.6,-10.2,-4.2,-2.3,-1.1,-2.6,11,-163.4,-153.2


In [659]:
if __name__=='__main__':
    s2 = getDriverSplitsReport(conn, rally, 'SS11', 'LAT', rc, typ, 'overall')
    display(HTML(s2))

,Road Position,1,2,3,SS11 Overall,D1,D2,D3,D4,Overall Position,Overall Time,Previous
LAT,8,,,,,,,,,1,,
SOR,11,-0.1,-2.6,-2.7,-4.2,-0.1,-2.5,-0.1,-1.5,2,-3.9,0.3
EVA,10,-0.7,-2,-3,-3,-0.7,-1.3,-1,,3,-11.3,-8.3
LOE,9,-1.3,-3.7,-2.9,-3.7,-1.3,-2.5,0.9,-0.8,4,-14.5,-10.8
OGI,6,-2.1,-1.7,-1.8,-1.9,-2.1,0.4,-0.1,-0.1,5,-14.7,-12.8
NEU,4,0.3,2,3.3,2.6,0.3,1.6,1.4,-0.7,6,-29.5,-32.1
LAP,3,-2.1,-0.7,-0.8,0.2,-2.1,1.3,-0.1,1,7,-37.6,-37.8
TÄN,12,-0.6,-0.3,0.1,1.4,-0.6,0.3,0.4,1.3,8,-60.4,-61.8
BRE,5,-1.6,-2,-2.1,-3.1,-1.6,-0.4,-0.1,-1,9,-62.1,-59
MIK,7,-5.8,-9.6,-12.3,-13.7,-5.8,-3.9,-2.6,-1.4,10,-93.1,-79.4


In [660]:
if __name__=='__main__':
    s2 = getDriverSplitsReport(conn, rally, 'SS11', 'LAT', rc, typ, 'previous')
    display(HTML(s2))

,Road Position,1,2,3,SS11 Overall,D1,D2,D3,D4,Overall Position,Overall Time,Previous
SOR,11,-0.1,-2.6,-2.7,-4.2,-0.1,-2.5,-0.1,-1.5,2,-3.9,0.3
LAT,8,,,,,,,,,1,,
EVA,10,-0.7,-2,-3,-3,-0.7,-1.3,-1,,3,-11.3,-8.3
LOE,9,-1.3,-3.7,-2.9,-3.7,-1.3,-2.5,0.9,-0.8,4,-14.5,-10.8
OGI,6,-2.1,-1.7,-1.8,-1.9,-2.1,0.4,-0.1,-0.1,5,-14.7,-12.8
NEU,4,0.3,2,3.3,2.6,0.3,1.6,1.4,-0.7,6,-29.5,-32.1
LAP,3,-2.1,-0.7,-0.8,0.2,-2.1,1.3,-0.1,1,7,-37.6,-37.8
BRE,5,-1.6,-2,-2.1,-3.1,-1.6,-0.4,-0.1,-1,9,-62.1,-59
TÄN,12,-0.6,-0.3,0.1,1.4,-0.6,0.3,0.4,1.3,8,-60.4,-61.8
MIK,7,-5.8,-9.6,-12.3,-13.7,-5.8,-3.9,-2.6,-1.4,10,-93.1,-79.4


In [662]:
if __name__=='__main__':
    s2 = getDriverSplitsReport(conn, rally, 'SS11', 'LAT', rc, typ, 'roadpos')
    display(HTML(s2))

,Road Position,1,2,3,SS11 Overall,D1,D2,D3,D4,Overall Position,Overall Time,Previous
AL,1,-26,-52.9,-67.1,-72.9,-26,-26.9,-14.2,-5.8,21,-832.3,-759.4
SUN,2,-4.2,-6.5,-7.6,-10.2,-4.2,-2.3,-1.1,-2.6,11,-163.4,-153.2
LAP,3,-2.1,-0.7,-0.8,0.2,-2.1,1.3,-0.1,1,7,-37.6,-37.8
NEU,4,0.3,2,3.3,2.6,0.3,1.6,1.4,-0.7,6,-29.5,-32.1
BRE,5,-1.6,-2,-2.1,-3.1,-1.6,-0.4,-0.1,-1,9,-62.1,-59
OGI,6,-2.1,-1.7,-1.8,-1.9,-2.1,0.4,-0.1,-0.1,5,-14.7,-12.8
MIK,7,-5.8,-9.6,-12.3,-13.7,-5.8,-3.9,-2.6,-1.4,10,-93.1,-79.4
LAT,8,,,,,,,,,1,,
LOE,9,-1.3,-3.7,-2.9,-3.7,-1.3,-2.5,0.9,-0.8,4,-14.5,-10.8
EVA,10,-0.7,-2,-3,-3,-0.7,-1.3,-1,,3,-11.3,-8.3


In [520]:
if __name__=='__main__':
    display(rb2.style.bar())

,Road Position,1,2,3,4,5,6,SS13 Overall,D1,D2,D3,D4,D5,D6,D7,Overall Position,Overall Time,Previous
NEU,4,1.5,nan,1.3,1,1.2,1.3,0.9,1.5,-1.5,1.3,-0.3,0.2,nan,-0.4,5,-6.8,-7.7
OGI,6,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2,nan,nan
SUN,2,-1.4,-2,-1.6,-2.6,-2.3,-3.9,-5.7,-1.4,-0.6,0.4,-1,0.3,-1.6,-1.8,11,-182,-176.3
EVA,10,-2.5,-3.6,-6,-6.1,-6.4,-6.7,-7.6,-2.5,-1.1,-2.4,-0.1,-0.3,-0.2,-0.9,4,-4,3.6
LAT,8,-1.4,-2.3,-6.4,-7,-6.5,-7.8,-8.2,-1.4,-0.9,-4.1,-0.6,0.6,-1.3,-0.4,1,7.2,15.4
LOE,9,-4.2,-5.5,-6.5,-7.6,-9.2,-9.7,-10.6,-4.2,-1.3,-1,-1.1,-1.6,-0.5,-0.9,3,-2.7,7.9
TÄN,12,2.2,2,-9.7,-10.3,-9.6,-11,-11.5,2.2,-0.2,-11.7,-0.6,0.7,-1.4,-0.5,8,-56.9,-45.4
LAP,3,1.4,3.3,5.9,-12.7,-11.9,-13.1,-13.9,1.4,1.9,2.5,-18.5,0.7,-1.2,-0.8,7,-34.4,-20.5
BRE,5,-2.5,-5.4,-7.8,-9.1,-10.2,-13,-14.4,-2.5,-2.9,-2.4,-1.4,-1,-2.9,-1.4,9,-90.6,-76.2
MIK,7,-3.5,-6.4,-11.2,-13.9,-14.8,-18.3,-20.4,-3.5,-2.9,-4.8,-2.7,-0.9,-3.5,-2.1,10,-121.1,-100.7


Problem with the bars is that the range is different in each column; ideally we want the same range in each column; could do this with two dummy rows to force max and min values?

In [613]:
if __name__=='__main__':
    #Example for pandas issue https://github.com/pandas-dev/pandas/issues/21526
    import pandas as pd
    import numpy as np
    
    df=pd.DataFrame({'x1':list(np.random.randint(-10,10,size=10))+[-500,1000, -1000],
               'y1':list(np.random.randint(-5,5,size=13)),'y2':list(np.random.randint(-2,3,size=13)) })
    
    display(df.style.bar( align='zero', color=[ '#5fba7d','#d65f5f']))

,x1,y1,y2
0,-8,-1,-1
1,9,-3,-1
2,-3,3,1
3,-7,-1,-2
4,7,-2,0
5,6,1,0
6,-6,2,-1
7,7,1,-2
8,-9,3,-1
9,3,3,0


In [614]:
if __name__=='__main__':
    #clip lets us set a max limiting range although it means we lose the actual value?
    df['x2']= df['x1'].clip(upper=10, lower=-10)
    display(df.style.bar( align='zero', color=[ '#d65f5f','#5fba7d']))

,x1,y1,y2,x2
0,-8,-1,-1,-8
1,9,-3,-1,9
2,-3,3,1,-3
3,-7,-1,-2,-7
4,7,-2,0,7
5,6,1,0,6
6,-6,2,-1,-6
7,7,1,-2,7
8,-9,3,-1,-9
9,3,3,0,3


In [620]:
if __name__=='__main__':
    #for pandas 0.24 ? https://github.com/pandas-dev/pandas/pull/21548
    df['x2']= df['x1'].clip(upper=10, lower=-10)
    #Set axis=None for table wide range?
    #display(df.style.bar( align='zero', axis=None, color=[ '#d65f5f','#5fba7d']))
    